In [1]:
import tensorflow as tf
import os
import sys
import data_generation
import networks
import scipy.io as sio
import param
import util
import truncated_vgg
from keras.backend.tensorflow_backend import set_session
from keras.optimizers import Adam

/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def train(model_name, gpu_id):
    params = param.get_general_params()

    network_dir = params['model_save_dir'] + '/' + model_name

    if not os.path.isdir(network_dir):
        os.mkdir(network_dir)

    train_feed = data_generation.create_feed(params, params['data_dir'], 'train')

    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

    vgg_model = truncated_vgg.vgg_norm()
    networks.make_trainable(vgg_model, False)
    response_weights = sio.loadmat('../data/vgg_activation_distribution_train.mat')
    model = networks.network_posewarp(params)
    model.compile(optimizer=Adam(lr=1e-4), loss=[networks.vgg_loss(vgg_model, response_weights, 12)])

    #model.summary()
    n_iters = params['n_training_iter']

    for step in range(0, n_iters):
        x, y = next(train_feed)

        train_loss = model.train_on_batch(x, y)

        util.printProgress(step, 0, train_loss)

        if step > 0 and step % params['model_save_interval'] == 0:
            model.save(network_dir + '/' + str(step) + '.h5')

In [3]:
params = param.get_general_params()

In [4]:
params

{'IMG_HEIGHT': 256,
 'IMG_WIDTH': 256,
 'batch_size': 4,
 'data_dir': '/home/jarvislam1999/posewarp-cvpr2018/data',
 'limbs': [[0, 1],
  [2, 3],
  [3, 4],
  [5, 6],
  [6, 7],
  [8, 9],
  [9, 10],
  [11, 12],
  [12, 13],
  [2, 5, 8, 11]],
 'max_px_shift': 10,
 'max_rotate_degree': 5,
 'max_sat_factor': 0.05,
 'model_save_dir': '/home/jarvislam1999/posewarp-cvpr2018/models',
 'model_save_interval': 1000,
 'n_joints': 14,
 'n_limbs': 10,
 'n_training_iter': 200000,
 'obj_scale_factor': 1.14,
 'posemap_downsample': 2,
 'project_dir': '/home/jarvislam1999/posewarp-cvpr2018',
 'scale_max': 1.05,
 'scale_min': 0.9,
 'sigma_joint': 1.75,
 'test_interval': 500}

In [5]:
params = param.get_general_params()

In [6]:
model_name = 'vgg_100000.h5'

In [7]:
network_dir = params['model_save_dir'] + '/' + model_name

In [8]:
network_dir

'/home/jarvislam1999/posewarp-cvpr2018/models/vgg_100000.h5'

In [9]:
params['data_dir']

'/home/jarvislam1999/posewarp-cvpr2018/data'

In [10]:
os.path.isdir(params['model_save_dir'])

True

In [11]:
if not os.path.isdir(params['model_save_dir']):
    os.mkdir(params['model_save_dir'])

In [12]:
train_feed = data_generation.create_feed(params, params['data_dir'], 'train')

In [13]:
train_feed

<generator object warp_example_generator at 0x7f4965f8ef10>

In [14]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [16]:
vgg_model = truncated_vgg.vgg_norm()
#networks.make_trainable(vgg_model, False)
#response_weights = sio.loadmat('../data/vgg_activation_distribution_train.mat')
#model = networks.network_posewarp(params)
#model.compile(optimizer=Adam(lr=1e-4), loss=[networks.vgg_loss(vgg_model, response_weights, 12)])

Exception: URL fetch failure on https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [Errno 110] Connection timed out

In [18]:
from keras.applications.vgg19 import VGG19
from keras.layers import AveragePooling2D, Conv2D, Input
from keras.models import Model

In [ ]:
img_input = Input(shape=(256, 256, 3))

x1 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x2 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x1)
x3 = AveragePooling2D((2, 2), strides=(2, 2), name='block1_pool')(x2)

x4 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x3)
x5 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x4)
x6 = AveragePooling2D((2, 2), strides=(2, 2), name='block2_pool')(x5)

x7 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x6)
x8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x7)
x9 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x8)
x10 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x9)
x11 = AveragePooling2D((2, 2), strides=(2, 2), name='block3_pool')(x10)

x12 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x11)
x13 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x12)
x14 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x13)
x15 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x14)
x16 = AveragePooling2D((2, 2), strides=(2, 2), name='block4_pool')(x15)

x17 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x16)
x18 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x17)
x19 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x18)
x20 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x19)
x21 = AveragePooling2D((2, 2), strides=(2, 2), name='block5_pool')(x20)

model = Model(inputs=[img_input], outputs=[x1, x2, x4, x5, x7, x8, x9, x10, x12, x13, x14, x15])
model_orig = VGG19(weights='imagenet', input_shape=(256, 256, 3), include_top=False)

'''
for i in range(len(model.layers)):
    weights = model_orig.layers[i].get_weights()
    model.layers[i].set_weights(weights)
'''